In [63]:
import paramiko
from getpass import getpass

# Create the password prompt
thepass = getpass(prompt="Please enter your SSH password: ")

# Host information
host = "192.168.6.71"
port = 2222
username = "zachary.morin"
password = thepass

try:

    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host, port, username, password)

    """
    (stdin, stdout, stderr) = ssh.exec_command("ls")
    for line in stdout.readlines():
        print(line)
    """

except paramiko.AuthenticationException:
    print("Authentication failed")

In [92]:
# Start SFTP session
sftp = ssh.open_sftp()

# List of files to download
#files = ['/etc/hosts','/etc/passwd','/var/log/auth.log']

sftp.put('./kraken','/home/zachary.morin/kraken')

safe_processes = ["snapd"] #snapd

(stdin, stdout, stderr) = ssh.exec_command("sudo chmod +x /home/zachary.morin/kraken && sudo /home/zachary.morin/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin", get_pty = True)
stdin.write(password + '\n')
z = stdout.readlines()[1:]

sketchy_list = []
for x in range(1, len(z)):
    #print(z[x])
    if "[33mprocess[0m=" in z[x]:
        for words in safe_processes:
            if (("[33mprocess[0m="+words) in z[x]) == False:
                sketchy_list.append(z[x])
    """
    for words in safe_processes:
        if ("[33mprocess[0m="+words) in safe_processes:
            pass
    """ 
#print(sketchy_list)

sketchy_pids = []
for warns in sketchy_list:
    pid = warns.split(" ")
    pid = pid[8]
    pid = pid.split('=')
    pid = pid[1]
    sketchy_pids.append(pid)

lsof_results = []
for pids in sketchy_pids:
    (stdin, stdout, stderr) = ssh.exec_command("sudo lsof -p "+pids, get_pty = True)
    stdin.write(password + '\n')
    lsof_results.append(stdout.readlines()[2:])

#print(lsof_results)


REG_files = []
for result in lsof_results:
    list_test = result
    for x in range(0, len(list_test)):
        if "REG" in list_test[x]:
            REG_files.append(list_test[x])
#print(REG_files)

file_locations = []
for reg in REG_files:
    test = reg.split("/")
    #print(test)
    for x in range(1, len(test)):
        if x == 1:
            e = "/"+test[x]
        else:
            e = e + "/" + test[x]
    e = e[:-2]
    #test = test[23]
    file_locations.append(e)
print(file_locations)

for eachFile in file_locations:

    # Create a list from the filename
    x = eachFile.split("/")

    print(x)

    # -1 is used to pluck the last element in a list.
    filename = x[-1]
    #print(filename)
    # Local path to store files
    local_path = "remote_files/" + filename
    sftp.get(eachFile, local_path)
    #sftp.put(LOCALFILE, REMOTESIDE)
ssh.close()

#ssh.close()



['/usr/local/bin/egrep', '/usr/lib/x86_64-linux-gnu/libc-2.31.so', '/usr/lib/x86_64-linux-gnu/libpthread-2.31.so', '/usr/lib/x86_64-linux-gnu/ld-2.31.so', '/home/p.dalton/.d.log', '/home/p.dalton/.d.log', '/home/p.dalton/d.pid']
['', 'usr', 'local', 'bin', 'egrep']
['', 'usr', 'lib', 'x86_64-linux-gnu', 'libc-2.31.so']
['', 'usr', 'lib', 'x86_64-linux-gnu', 'libpthread-2.31.so']
['', 'usr', 'lib', 'x86_64-linux-gnu', 'ld-2.31.so']
['', 'home', 'p.dalton', '.d.log']
['', 'home', 'p.dalton', '.d.log']
['', 'home', 'p.dalton', 'd.pid']


In [42]:
# Run a single Command
#command = "ps -ef"

# Run a list of commands
command = ['ps -ef', 'netstat -an --inet','last -adx', 'cut -d: -f1 /etc/passwd']

for eachCMD in command:

    # Get the output from the command
    (stdin, stdout, stderr) = ssh.exec_command(eachCMD)

    # Get results from stdout
    lines = stdout.readlines()
    #print(lines)

    #Convert the list to a string
    output = ''.join(lines)

    # Header output
    sepHeader = '' + '### BEGIN ' + eachCMD + '###\n\n'

    # Footer
    sepFooter = '' + '### End ' + eachCMD + '###\n\n'
    cmd_output = sepHeader + output + sepFooter

    # Save cmd_output to a file
    with open('ssh_results.txt', 'a') as f:
        f.write(cmd_output)
    #print(cmd_output)

ssh.close()
